In [5]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.preprocessing import StandardScaler

In [7]:
df = pd.read_excel("../data/TCGA_MEASUREMENTS.xlsx")

In [8]:
df["years_to_event"] = df["days_to_event"]/365.25

In [9]:
X = df.drop(columns=["years_to_event", "days_to_event"])

In [14]:
X

,ID,ADI,BACK,DEB,LYM,MUC,MUS,NORM,STR,TUM,...,pathology_N_stage,pathology_M_stage,gender,histological_type,hypermutated,methylation_subtype,CAF_SCORE,percent_stromal_cells,RF_predictedCMS,cleanstage
0,TCGA-CM-6675,0.000284,0.000204,0.090979,0.000544,0.013228,0.007923,0.101498,0.086859,0.798861,...,n2b,m1b,male,colon adenocarcinoma,0.0,CIMP-H,2.080628,12,CMS1,4.0
1,TCGA-AY-A8YK,0.000324,0.000290,0.004827,0.013253,0.004651,0.002694,0.066702,0.140175,0.767085,...,n2a,m1,male,colon adenocarcinoma,NaN,NaN,1.635184,7,NaN,4.0
2,TCGA-CM-4747,0.001219,0.004085,0.197126,0.337597,0.002646,0.003099,0.158479,0.447476,0.485696,...,n1b,m1a,male,colon adenocarcinoma,0.0,CIMP-L,2.024608,15,NaN,4.0
3,TCGA-DY-A1DG,0.003772,0.001362,0.188463,0.002173,0.012698,0.063342,0.096374,0.012094,0.619722,...,n1,m1,male,rectal adenocarcinoma,NaN,Cluster3,0.990050,0,NaN,4.0
4,TCGA-CM-5862,0.007687,0.006287,0.386051,0.086920,0.132158,0.130372,0.076991,0.102107,0.481279,...,n1a,m1a,male,colon adenocarcinoma,0.0,Cluster3,1.944954,0,CMS2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,TCGA-CL-4957,0.008215,0.031160,0.078296,0.023389,0.009071,0.032259,0.202351,0.413524,0.201735,...,n1,m0,female,rectal adenocarcinoma,0.0,Cluster4,2.032620,20,CMS2,NaN
496,TCGA-AD-6890,0.000167,0.000214,0.022478,0.002921,0.031190,0.001960,0.091274,0.326789,0.523009,...,n0,mx,male,colon adenocarcinoma,0.0,CIMP-H,2.115697,10,CMS2,NaN
497,TCGA-AH-6644,0.000707,0.001519,0.301201,0.178563,0.042361,0.005472,0.070207,0.074859,0.325110,...,nx,mx,male,rectal adenocarcinoma,0.0,Cluster4,3.260665,25,CMS4,NaN
498,TCGA-AD-6963,0.000042,0.000178,0.001873,0.056791,0.000457,0.000084,0.193539,0.001077,0.745959,...,n0,mx,male,colon adenocarcinoma,NaN,Cluster4,1.657536,25,CMS2,NaN


In [17]:
X[["ADI","BACK", "DEB", "LYM", "MUC","MUS", "NORM", "STR","TUM"]]

,ADI,BACK,DEB,LYM,MUC,MUS,NORM,STR,TUM
0,0.000284,0.000204,0.090979,0.000544,0.013228,0.007923,0.101498,0.086859,0.798861
1,0.000324,0.000290,0.004827,0.013253,0.004651,0.002694,0.066702,0.140175,0.767085
2,0.001219,0.004085,0.197126,0.337597,0.002646,0.003099,0.158479,0.447476,0.485696
3,0.003772,0.001362,0.188463,0.002173,0.012698,0.063342,0.096374,0.012094,0.619722
4,0.007687,0.006287,0.386051,0.086920,0.132158,0.130372,0.076991,0.102107,0.481279
...,...,...,...,...,...,...,...,...,...
495,0.008215,0.031160,0.078296,0.023389,0.009071,0.032259,0.202351,0.413524,0.201735
496,0.000167,0.000214,0.022478,0.002921,0.031190,0.001960,0.091274,0.326789,0.523009
497,0.000707,0.001519,0.301201,0.178563,0.042361,0.005472,0.070207,0.074859,0.325110
498,0.000042,0.000178,0.001873,0.056791,0.000457,0.000084,0.193539,0.001077,0.745959


In [21]:
scaled_X = X[["ADI","BACK", "DEB", "LYM", "MUC","MUS", "NORM", "STR","TUM"]].copy()
col_names = scaled_X.columns
features = scaled_X[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_X[col_names] = features

In [30]:
df[["years_to_event","vital_status"]]

,years_to_event,vital_status
0,1.086927,0
1,1.568789,0
2,2.083504,0
3,4.287474,1
4,0.418891,1
...,...,...
495,0.000000,1
496,2.042437,0
497,2.294319,0
498,2.283368,0


In [35]:
y = df[["years_to_event","vital_status"]]
y_days = df[["days_to_event","vital_status"]]
scaled_X

,ADI,BACK,DEB,LYM,MUC,MUS,NORM,STR,TUM
0,-0.192161,-0.280475,-0.047387,-0.811732,-0.300486,-0.239747,-0.316563,-0.207776,0.747461
1,-0.191168,-0.276241,-0.730799,-0.746327,-0.377028,-0.293643,-0.539877,0.182238,0.617916
2,-0.169236,-0.089033,0.794650,0.922924,-0.394922,-0.289460,0.049134,2.430197,-0.529237
3,-0.106658,-0.223351,0.725926,-0.803352,-0.305214,0.331416,-0.349448,-0.754687,0.017155
4,-0.010718,0.019602,2.293340,-0.367195,0.760925,1.022235,-0.473846,-0.096231,-0.547245
...,...,...,...,...,...,...,...,...,...
495,0.002229,1.246650,-0.147991,-0.694162,-0.337578,0.011059,0.330693,2.181834,-1.686875
496,-0.195029,-0.280001,-0.590783,-0.799503,-0.140180,-0.301207,-0.382179,1.547351,-0.377119
497,-0.181778,-0.215589,1.620248,0.104446,-0.040482,-0.265008,-0.517379,-0.295552,-1.183906
498,-0.198088,-0.281767,-0.754233,-0.522257,-0.414462,-0.320537,0.274140,-0.835281,0.531794


In [64]:
cph = CoxPHFitter().fit(pd.concat([df["BACK"], y], axis=1), "years_to_event", "vital_status" )

In [65]:
cph.print_summary()

/home/bartek/miniconda3/envs/dslab/lib/python3.9/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 500 total observations, 394 right-censored observations>
             duration col = 'years_to_event'
                event col = 'vital_status'
      baseline estimation = breslow
   number of observations = 500
number of events observed = 106
   partial log-likelihood = -573.23
         time fit was run = 2022-11-12 10:16:52 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
BACK       -4.23       0.01       5.10           -14.23             5.78                 0.00               322.31

            cmp to     z    p   -log2(p)
covariate                               
BACK          0.00 -0.83 0.41       1.30
---
Concordance = 0.46
Partial AIC = 1148.47
log-likelihood ratio test = 0.96 on 1 df
-log2(p) of ll-ratio test = 1.61